In [2]:
from rdflib import Graph


In [9]:


# Create a Graph object
g = Graph()

# Load RDF data from a file
g.parse("pg1.rdf", format="xml")  # Replace "xml" with the appropriate format if needed


<Graph identifier=N1bddbb40f5ed4d959710ba42b34f1713 (<class 'rdflib.graph.Graph'>)>

In [11]:
len(g)

204

In [12]:
from rdflib import Graph, Namespace

# Load the RDF graph
g = Graph()
g.parse("pg1.rdf", format="xml")  # Replace with the path to your RDF file

# Define namespaces
DCTERMS = Namespace("http://purl.org/dc/terms/")
PGTERMS = Namespace("http://www.gutenberg.org/2009/pgterms/")

# Extract Title
title = None
for t in g.objects(None, DCTERMS.title):
    title = t
    break  # Assume only one title

# Extract Author
author_name = None
for agent in g.subjects(DCTERMS.creator, None):
    for name in g.objects(agent, PGTERMS.name):
        author_name = name
        break  # Assume only one author

# Extract Descriptions
descriptions = []
for desc in g.objects(None, DCTERMS.description):
    descriptions.append(desc)

# Print the extracted data
print(f"Title: {title}")
print(f"Author: {author_name}")
print("Descriptions:")
for description in descriptions:
    print(f"- {description}")


Title: The Declaration of Independence of the United States of America
Author: None
Descriptions:
- Wikipedia page about this book: https://en.wikipedia.org/wiki/United_States_Declaration_of_Independence
- This is the original PG edition.
See also our revised edition: #16780
See also #300
- Reading ease score: 52.1 (10th to 12th grade). Somewhat difficult to read.
- en.wikipedia


In [13]:
pwd


'c:\\Users\\gbernar\\OneDrive - MORNINGSTAR INC\\HF'

In [14]:
import os
import xml.etree.ElementTree as ET

# Base directory where the folders are located
base_dir = 'c:\\Users\\gbernar\\OneDrive - MORNINGSTAR INC\\HF'

# List to store extracted data
data = []

# Iterate over folder numbers
for folder_number in range(1, 2):  # 1 to 90907 inclusive
    folder_path = os.path.join(base_dir, str(folder_number))
    for file_number in range(1, 2):  # 1 to 90907 inclusive
        file_name = f"pg{file_number}.rdf"
        file_path = os.path.join(folder_path, file_name)

        if not os.path.isfile(file_path):
            continue  # Skip if the file does not exist

        try:
            # Parse the RDF file
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract relevant information
            author = root.find(".//pgterms:agent/pgterms:name", namespaces=root.nsmap)
            title = root.find(".//dcterms:title", namespaces=root.nsmap)
            description_elements = root.findall(".//dcterms:description", namespaces=root.nsmap)
            value_elements = root.findall(".//rdf:value", namespaces=root.nsmap)

            # Handle cases where the tags may be missing
            author = author.text if author is not None else None
            title = title.text if title is not None else None
            descriptions = [desc.text for desc in description_elements if desc.text]
            values = [val.text for val in value_elements if val.text]

            # Append extracted information to the data list
            data.append({
                "author": author,
                "title": title,
                "descriptions": descriptions,
                "values": values
            })
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Now `data` contains the list of extracted information
# You can save or process it further as needed


Error processing file c:\Users\gbernar\OneDrive - MORNINGSTAR INC\HF\1\pg1.rdf: 'xml.etree.ElementTree.Element' object has no attribute 'nsmap'


In [ ]:
import os
import xml.etree.ElementTree as ET

# Namespace mapping
namespaces = {
    'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'dcterms': 'http://purl.org/dc/terms/',
    'pgterms': 'http://www.gutenberg.org/2009/pgterms/'
}

# Data storage
data = []

# Loop through folders and files
for folder_number in range(1, 2):
    folder_name = str(folder_number)
    for file_number in range(1,2):
        file_name = f"pg{file_number}.rdf"
        file_path = os.path.join(folder_name, file_name)
        
        if os.path.exists(file_path):
            try:
                tree = ET.parse(file_path)
                root = tree.getroot()

                # Extract data
                title = root.find('.//dcterms:title', namespaces)
                description = root.find('.//dcterms:description', namespaces)
                creator = root.find('.//dcterms:creator/pgterms:agent/pgterms:name', namespaces)
                subjects = root.findall('.//dcterms:subject/rdf:Description/rdf:value', namespaces)

                # Store the extracted data
                record = {
                    'title': title.text if title is not None else None,
                    'description': description.text if description is not None else None,
                    'author': creator.text if creator is not None else None,
                    'subjects': [subject.text for subject in subjects if subject is not None]
                }
                data.append(record)
            
            except ET.ParseError as e:
                print(f"Failed to parse {file_path}: {e}")
            except Exception as e:
                print(f"Error processing {file_path}: {e}")

# Now, `data` contains all the extracted information.


In [18]:
data

[{'title': 'The Declaration of Independence of the United States of America',
  'description': 'Wikipedia page about this book: https://en.wikipedia.org/wiki/United_States_Declaration_of_Independence',
  'author': 'Jefferson, Thomas',
  'subjects': ['United States -- History -- Revolution, 1775-1783 -- Sources',
   'United States. Declaration of Independence',
   'E201',
   'JK']}]

In [ ]:
data